In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
import torch
import numpy as np
import re
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings("ignore")

In [3]:
model_type = 'xlm-roberta-base' #albert-base-v1, bert-base-cased, bert-base-uncased
data_path = "../dataset/zh-en/"

with open(data_path + 'train_texts_zh.txt', 'r', encoding='utf-8') as f:
    train_text = f.readlines()
with open(data_path + 'dev_texts_zh.txt', 'r', encoding='utf-8') as f:
    valid_text = f.readlines()
with open(data_path + 'test_texts_zh.txt', 'r', encoding='utf-8') as f:
    test_text = f.readlines()

In [4]:
datasets = train_text, valid_text, test_text

In [5]:
[len(ds) for ds in datasets]

[1017, 8, 11]

In [6]:
def clean_text(text):
    text = text.replace('！', '。')
    text = text.replace('：', '，')
    text = text.replace('——', '，')
    
    #reg = "(?<=[a-zA-Z])-(?=[a-zA-Z]{2,})"
    #r = re.compile(reg, re.DOTALL)
    #text = r.sub(' ', text)
    
    text = re.sub(r'\s—\s', ' ， ', text)
    
#     text = text.replace('-', ',')
    text = text.replace(';', '。')    # replace symbols with the most relevant counterparts
    text = text.replace('、', '，')
    text = text.replace('♫', '')
    text = text.replace('……', '')
    text = text.replace('。”', '')
    text = text.replace('”', '，')
    text = text.replace('“','，')
    text = text.replace(',','，')
    

    text = re.sub(r'——\s?——', '', text) # replace --   -- to ''
    text = re.sub(r'\s+', ' ', text)    # strip all whitespaces
    
    text = re.sub(r'，\s?，', '，', text)  # merge commas separating only whitespace
    text = re.sub(r'，\s?。', '。', text) # , . -> ,
    text = re.sub(r'？\s?。', '？', text)# ? . -> ?
    text = re.sub(r'\s+', ' ', text)    # strip all redundant whitespace that could have been caused by preprocessing
    
    text = re.sub(r'\s+？', '？', text)
    text = re.sub(r'\s+，', '，', text)
    text = re.sub(r'。[\s+。]+', '。 ', text)
    text = re.sub(r'\s+。', '。 ', text)
    
    return text.strip().lower()

In [7]:
datasets = [[clean_text(text) for text in ds] for ds in datasets]

In [8]:
[len([t for t in ds if len(t)>0]) for ds in datasets] # remove all 0 word datasets

[1017, 8, 11]

In [9]:
[len(' '.join(ds).split(' ')) for ds in datasets] # make them sentences separated by a space for tokenizing

[316676, 2307, 3608]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_type)

In [11]:
target_ids = tokenizer.encode("。？，")[1:-1]
tokenizer.convert_ids_to_tokens(target_ids)

['▁', '。', '?', ',']

In [12]:
target_token2id = {t: tokenizer.encode(t)[-2] for t in "。?,"}
target_token2id

{'。': 30, '?': 705, ',': 4}

In [13]:
target_ids = list(target_token2id.values())
target_token2id.items()
#target_ids

dict_items([('。', 30), ('?', 705), (',', 4)])

In [42]:
id2target = {
    0: 0,
    -1: -1,
}
for i, ti in enumerate(target_ids):
    id2target[ti] = i+1
target2id = {value: key for key, value in id2target.items()}
print(id2target, target2id)
def create_target(text):
    encoded_words, targets = [], []
    
    words = tokenizer.tokenize(text)[1:] ## ignore the first space

    words2 = []
    for i in range(len(words)):
        if words[i] not in ["。","?",","," ","▁"]:
            if i < len(words) -1 and words[i+1] in ["。","?",","," ","▁"]:
                words2.append(words[i])
            else:
                words2.extend([words[i],' '])
        else:
            if words[i] == "▁":
                if i > 0 and words[i-1] not in ["。","?",","," ","▁"]:
                    words2.append(" ")
            else:
                words2.append(words[i])
            
    words = words2
    
    for word in words:
        target = 0
        target_appended = False
        for target_token, target_id in target_token2id.items():
            if word == target_token:
                #word = word.rstrip(target_token)
                encoded_words.append(target_token2id[word])
                targets.append(id2target[target_id])
                target_appended = True
        if not target_appended:
            if word == ' ':
                encoded_words.append(6)
                targets.append(0)
            else:    
                encoded_word = tokenizer.encode(word, add_special_tokens=False)

                if len(encoded_word) == 2:
                    encoded_word = encoded_word[1:]

                for w in encoded_word:
                    encoded_words.append(w)

                if len(encoded_word)>1:
                    for _ in range(len(encoded_word)-1):
                        if encoded_word[_] == 6:
                            targets.append(0)
                        else:
                            targets.append(-1)
                    targets.append(0)
                else:
                    targets.append(0)    

#             print([tokenizer._convert_id_to_token(ew) for ew in encoded_word], target)
            assert(len(encoded_word)>0)
    
    encoded_words = [tokenizer.cls_token_id or tokenizer.bos_token_id] +\
                    encoded_words +\
                    [tokenizer.sep_token_id or tokenizer.eos_token_id]
    
    targets = [-1] + targets + [-1]
    
    return encoded_words, targets

{0: 0, -1: -1, 30: 1, 705: 2, 4: 3} {0: 0, -1: -1, 1: 30, 2: 705, 3: 4}


In [43]:
print(id2target)
s = "谁能猜一猜：你大脑里神经元的总长有多少？ ”西班牙厨师被控告……“ 非常坚硬的土地。西班牙厨师被控告"
print(s)
s = clean_text(s)
print(s)
data, targets = create_target(s)
print(data)
print(targets)
[(tokenizer._convert_id_to_token(d), ta) for d,ta in zip(data[2:-1], targets[2:-1])]

{0: 0, -1: -1, 30: 1, 705: 2, 4: 3}
谁能猜一猜：你大脑里神经元的总长有多少？ ”西班牙厨师被控告……“ 非常坚硬的土地。西班牙厨师被控告
谁能猜一猜，你大脑里神经元的总长有多少？，西班牙厨师被控告， 非常坚硬的土地。西班牙厨师被控告
[0, 19874, 6, 1580, 6, 84952, 6, 45690, 6, 84952, 4, 73675, 6, 157938, 6, 2008, 6, 133614, 6, 112535, 6, 7051, 6, 3846, 6, 138561, 705, 4, 54222, 6, 195223, 6, 17061, 6, 1317, 6, 17154, 6, 22292, 4, 4528, 6, 73613, 6, 21344, 6, 43, 6, 20770, 30, 54222, 6, 195223, 6, 17061, 6, 1317, 6, 17154, 6, 22292, 6, 2]
[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]


[('▁', 0),
 ('能', 0),
 ('▁', 0),
 ('猜', 0),
 ('▁', 0),
 ('▁一', 0),
 ('▁', 0),
 ('猜', 0),
 (',', 3),
 ('▁你', 0),
 ('▁', 0),
 ('大脑', 0),
 ('▁', 0),
 ('里', 0),
 ('▁', 0),
 ('神经', 0),
 ('▁', 0),
 ('元的', 0),
 ('▁', 0),
 ('总', 0),
 ('▁', 0),
 ('长', 0),
 ('▁', 0),
 ('有多少', 0),
 ('▁?', 2),
 (',', 3),
 ('西班牙', 0),
 ('▁', 0),
 ('厨', 0),
 ('▁', 0),
 ('师', 0),
 ('▁', 0),
 ('被', 0),
 ('▁', 0),
 ('控', 0),
 ('▁', 0),
 ('告', 0),
 (',', 3),
 ('非常', 0),
 ('▁', 0),
 ('坚', 0),
 ('▁', 0),
 ('硬', 0),
 ('▁', 0),
 ('的', 0),
 ('▁', 0),
 ('土地', 0),
 ('。', 1),
 ('西班牙', 0),
 ('▁', 0),
 ('厨', 0),
 ('▁', 0),
 ('师', 0),
 ('▁', 0),
 ('被', 0),
 ('▁', 0),
 ('控', 0),
 ('▁', 0),
 ('告', 0),
 ('▁', 0)]

In [ ]:
# encoded_texts, targets = create_target(transcripts[164])

In [ ]:
# print(datasets[0][0])

In [44]:
encoded_texts, targets = [], []

for ds in datasets:
    trgts = []
    for ts in ds:
        trgts.extend([ts[i:i+512] for i in range(0,len(ts),512)])
    x = list(zip(*(create_target(trgt) for trgt in tqdm(trgts))))
    encoded_texts.append(x[0])
    targets.append(x[1])

  0%|          | 0/8816 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

In [ ]:
# encoded_words, targets
comma_count = 0
word_count = 0
q_count = 0
p_count = 0

for tar in targets[0]:
    for ta in tar:
        comma_count += 1 if (ta == 3) else 0
        word_count += 1 if (ta != -1) else 0
        q_count += 1 if (ta == 2) else 0
        p_count += 1 if (ta == 1) else 0
        
print(comma_count, word_count, q_count, p_count)
            

'''
for te, ta in zip(encoded_texts[0][0], targets[0][0]):
    print(f"{tokenizer._convert_id_to_token(te):15}\t{ta}")
'''

188165 2339461 10215 139619


'\nfor te, ta in zip(encoded_texts[0][0], targets[0][0]):\n    print(f"{tokenizer._convert_id_to_token(te):15}\t{ta}")\n'

In [45]:
os.makedirs(data_path + model_type, exist_ok=True)

for i, name in enumerate(('train', 'valid', 'test')):
    with open(data_path + f'{model_type}/{name}_data.pkl', 'wb') as f:
        pickle.dump((encoded_texts[i], targets[i]), f)

9004


In [ ]:
from collections import Counter

for ds_targets in targets:
    c = Counter((target for t in ds_targets for target in t))
    print('\t'.join([str(c[i]) for i in (1,2,3,0,-1)]))

139619	10215	188165	2001462	267423
909	71	1225	15141	1899
1100	46	1120	16208	2072


In [ ]:
e = []
i = 0

raw_words = datasets[1][2].split(' ')

for te, ta in zip(encoded_texts[1][2], targets[1][2]):
    if ta == -1:
        e.append(te)
    else:
        e.append(te)
        print(f"{tokenizer.decode(e):15}\t{tokenizer.decode(target2id[ta]):10}\t{raw_words[i]}")
        e = []
        i += 1
print(f"{tokenizer.decode(e):15}\t{tokenizer.decode(target2id[ta]):10}\t")

In [33]:
# print(tokenizer.decode(encoded_texts[0][5]))
print(targets[0][5])

<s>小孩 的 母亲 们,应该 买 她们 认为 健康 的东西, 但 这些 东西 实际上 有 毒。 这 导致 了  日本 一系列 的 其他 运动。 在 这一点 上,我真的 非常 骄傲 的 说,在日本 买 任何 东西 都 很难 这是 标签 贴 错了,即使 他们 仍然  在 出售 鲸 肉, 而 我认为 他们 不 该 这么 做。 但是 至少 标签 帖 对 了 你就 不 回 再 去 买 有 毒 的 海 豚 肉。并不是 只有  日本 才 这样,而是  在 一些 国家的 自然 食物 链 都 这样  在 加拿大 北部,在美国 还有 欧洲 北部,海 豹 和 鲸 鱼 的 自然 食物 链 导致 了  pc  b 分子 的 富 集  从 世界上 的 各个 地方 聚集 到 妇女 的 身上。这些 妇女 的 乳 汁 含 毒。她们 不能 用 她们 的 乳 汁 来 喂 养 她们 的 孩子们 因为 富 集 的 毒 素  在 她们 的食物 链 之中, 在 她们 世界中 的一部分  在 海洋 金 字 塔 食物 链 里。 这 说明 她们 的 免疫 系统 已经 受到 危害。 这 说明 她们 后 代 的 生长 发育 也会 受到 危害。近 十年 世界上 对 这一 问题的 关注 已经 帮助 这些 妇女 解决 了 这个问题,不是 通过 改变 食物 链 结构,而是 改变 她们 特 有的 饮食。 我们 已经 让 这些 妇女 脱离 自然 的食物 链 目的 就是 解决 这个问题。 对于 这个 特别 尖 锐 的问题,这是 个 好 办法, 但 它 对 解决 金 字 塔 食物 链 问题 没什么 帮助。还有 另一种 方法 打破 金 字 塔 食物 链。 如果 我们在 金 字 塔 食物 链 底 部 塞 入 一些 东西,</s>
[-1, 42328, 0, 43, 0, 49889, 0, 9144, 4, 10730, 0, 11795, 0, 76026, 0, 9413, 0, 6278, 0, 39151, 4, 53072, 0, 5001, 0, 25840, 0, 59814, 0, 465, 0, 20291, 30, 63982, 0, 24754, 0, 274, 0, 34891, 0, 42652, 0, 43, 0, 5610, 0, 28188, 30, 6376, 0, 94772, 0, 575, 4, 